In [3]:
import numpy as np
from sys import path
path.append('../tools')
from tools import logsig

In [4]:
def get_expected_output(X):
    return logsig(np.dot(logsig(X * np.array([10, 10]) + np.array([-5, 5])), np.array([1, 1])) + -1)

In [5]:
vec_get_y = np.vectorize(get_expected_output)

In [6]:
x_train = np.linspace(-2, 2, 82)
y_train = vec_get_y(x_train)

In [7]:
x_train = x_train.reshape(len(x_train), 1)
y_train = y_train.reshape(len(y_train), 1)

In [8]:
# Create the model with keras and tensorflow
# Adapted from : https://keras.io/guides/custom_train_step_in_tensorflow/

# Imports. Need to do this environment stuff for some reason
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

import tensorflow as tf
import keras
from keras import layers

In [36]:
# Create the model
class CustomModel(keras.Model):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.search_direction = None
        self.f_search_direction = None
        self.beta = None
        self.old_f_gradients = None
    
    def update_weights(self, lr, weights):
        for i in range(len(self.trainable_variables)):
            self.trainable_variables[i].assign(weights[i] + lr*self.search_direction[i])
            
    def train_step(self, data):
        self.optimizer.learning_rate.assign(.1)
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compute_loss(y=y, y_pred=y_pred)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        f_gradients = tf.concat([tf.reshape(gradient, [-1]) for gradient in gradients], 0)

        # Get the search direction
        if self.search_direction is None:
            self.search_direction = [- x for x in gradients]
            self.f_search_direction = -f_gradients
            self.old_f_gradients = f_gradients
    
        if self.search_direction is not None:
            gradient_delta = f_gradients - self.old_f_gradients
            self.beta = tf.tensordot(gradient_delta, f_gradients, 1) / tf.tensordot(gradient_delta, self.f_search_direction, 1)
            self.f_search_direction = -f_gradients + self.beta * self.f_search_direction
            
            # Unflatten the search direction
            # Step 1: get the amount of variables needed
            # for weight/bias matrix
            shapes = [x.shape for x in gradients]
            sizes = []
            for shape in shapes:
                prod = 1
                for size in shape:
                    prod *= size
                sizes.append(prod)
            
            # Step 2: create the unflattened tensors
            # and replace in search direction
            start_slice = 0
            end_slice = 0
            for i in range(len(shapes)):
                end_slice += sizes[i]
                self.search_direction[i] = tf.reshape(self.f_search_direction[start_slice:end_slice], shapes[i])
                start_slice += sizes[i]
            
        # Find interval for the best learning rate
        if self.beta is not None:
            # Interval search
            # Initial guess for learning rate
            a = 0.01
            # Update the weights and evaluate loss
            self.update_weights(a, trainable_vars)
            e_a = self.compute_loss(y=y, y_pred=self(x, training=True))

            # Get second bound of initial guess
            b = 2*a
            self.update_weights(b, trainable_vars)
            e_b = self.compute_loss(y=y, y_pred=self(x, training=True))

            for i in range(10000):
                # Update the variables
                # Keep track of furthest back
                old_a = a
                old_e_a = e_a
                # Middle bound
                a = b
                e_a = e_b
                # Furthest
                b = 2*b
                self.update_weights(b, trainable_vars)
                e_b = self.compute_loss(y=y, y_pred=self(x, training=True))

                # Check whether the error has increased
                if e_b > e_a:
                    a = old_a
                    e_a = old_e_a
                    # We found the interval, break
                    print('Interval found')
                    break
                    

        # Update weights
        # self.optimizer.apply([-x for x in self.search_direction], trainable_vars)

        # Update metrics (includes the metric that tracks the loss)
        for metric in self.metrics:
            if metric.name == "loss":
                metric.update_state(loss)
            else:
                metric.update_state(y, y_pred)

        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

SyntaxError: invalid syntax (821921886.py, line 91)

In [37]:
# Construct and compile an instance of CustomModel
inputs = keras.Input(shape=(1,))
x = keras.layers.Dense(2)(inputs)
outputs = keras.layers.Dense(1)(x)
model = CustomModel(inputs, outputs)
model.compile(optimizer="SGD", loss="mse")

model.fit(x_train, y_train, epochs=3)

Epoch 1/3


OperatorNotAllowedInGraphError: Using a symbolic `tf.Tensor` as a Python `bool` is not allowed. You can attempt the following resolutions to the problem: If you are running in Graph mode, use Eager execution mode or decorate this function with @tf.function. If you are using AutoGraph, you can try decorating this function with @tf.function. If that does not work, then you may be using an unsupported feature or your source code may not be visible to AutoGraph. See https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/limitations.md#access-to-source-code for more information.